# TFF Skin Lesion Type Classification
## Transforming the Data into Federated Format

---

This notebook works through the the ETL process, to package the data into the federated format.

---

**Original data source:**
The HAM10000 dataset served as the training set for the [ISIC 2018 challenge (Task 3)](https://arxiv.org/abs/1902.03368) back in 2018. The official validation- and test-sets of this challenge are available, without ground-truth labels, through the challenge website https://challenge2018.isic-archive.com/. 

**Data import source for this notebook:**
The HAM10000 data used in this notebook is loaded via the Kaggle API from this [Kaggle data source](https://www.kaggle.com/kmader/skin-cancer-mnist-ham10000).

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T

---

The author of this notebook used code snipets from [K Scott Mader](https://www.kaggle.com/kmader) and [Manu Siddhartha](https://www.kaggle.com/sid321axn).


## 1. Installations
Google Colab is a Jupyter notebook environment that runs entirely in the cloud. Usually, the environment is already setup with TensorFlow 2. However, the author installs the necessary libraries manually, to make sure the required versions are available in the runtime environment.


In [1]:
!pip install tensorflow==2.2.0rc1

     |████████████████████████████████| 516.1MB 26kB/s 
     |████████████████████████████████| 3.9MB 49.1MB/s 
  Found existing installation: tensorboard 2.2.1
    Uninstalling tensorboard-2.2.1:
      Successfully uninstalled tensorboard-2.2.1
  Found existing installation: tensorflow 2.2.0rc3
    Uninstalling tensorflow-2.2.0rc3:
      Successfully uninstalled tensorflow-2.2.0rc3


In [0]:
!pip install -q kaggle

In [3]:
!pip install plotly==4.6.0

     |████████████████████████████████| 7.2MB 3.4MB/s 
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [4]:
!pip install scikit-image==0.16.2

In [5]:
!pip install imageio

In [6]:
!pip install imread

     |████████████████████████████████| 1.6MB 3.4MB/s 


In [7]:
!pip install numpy

### 1.1 Check the installed version

In [8]:
import tensorflow as tf
print(tf.__version__)

2.2.0-rc1


In [9]:
import numpy as np
print(np.__version__)

1.18.3


### 1.2 Setting up the Kaggle API
Using the Kaggle API on Google Colab allows you to directly work with the dataset without downloading and uploading it through your local machine. BUT the disadvantage of this approach is that every session you use Colab, the downloaded data sets and the kaggle json file will be gone and will have to be manually downloaded again.

In [10]:
# mount your google drive so you can save to it. You'll need to put in a token.
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
from google.colab import files
#files.upload()

In [0]:
# create environment variables for kaggle to authenticate with
import os

os.environ['KAGGLE_USERNAME'] = "christinasalker"
os.environ['KAGGLE_KEY'] = "personal_key"

In [13]:
# let's list what's in the directory
os.listdir()

['.config', 'gdrive', 'kaggle.json', 'sample_data']

In [0]:
# make directory named kaggle and copy kaggle.json file there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [15]:
# let's make a new directory for c_skin
os.mkdir('c_skin')
os.listdir()

['.config', 'gdrive', 'c_skin', 'kaggle.json', 'sample_data']

## 2. Extract Data

### 2.1 Download and unzip HAM10000 to local directory

In [16]:
# get the dataset from kaggle and load it into c_skin
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 -p 'c_skin'

100% 5.19G/5.20G [01:29<00:00, 40.7MB/s]
100% 5.20G/5.20G [01:29<00:00, 62.3MB/s]


In [17]:
# unzip the file into /c_skin
!unzip -o c_skin/skin-cancer-mnist-ham10000.zip -d c_skin

Streaming output truncated to the last 5000 lines.
  inflating: c_skin/ham10000_images_part_2/ISIC_0029326.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029327.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029328.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029329.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029330.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029331.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029332.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029333.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029334.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029335.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029336.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029337.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029338.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029339.jpg  
  inflating: c_skin/ham10000_images_part_2/ISIC_0029340.jpg  
  inflating: c_skin

In [18]:
# run this command to see all the files unzipped into the c_skin directory
!ls c_skin

ham10000_images_part_1	HAM10000_metadata.csv  hmnist_8_8_RGB.csv
HAM10000_images_part_1	hmnist_28_28_L.csv     skin-cancer-mnist-ham10000.zip
ham10000_images_part_2	hmnist_28_28_RGB.csv
HAM10000_images_part_2	hmnist_8_8_L.csv


### 2.2 Create df with image path

In [19]:
# first import the usual frameworks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import warnings
import json
import os
from glob import glob

from IPython.core.display import display, HTML

# import plotly 
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls

# for color scales in plotly
import colorlover as cl 

# for processing images
from skimage import data, io, filters

# configure things
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.2f}'.format  
pd.options.display.max_columns = 999

py.init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
# building the path column for the individual images is a bit tricky 
# first the base_skin_dir which will be used for the imageid_path_dic needs to be defined
base_skin_dir = os.path.join('c_skin')

In [0]:
# this os.path method is used to create the image path
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x
                     for x in glob(os.path.join(base_skin_dir, '*', '*.jpg'))}

In [22]:
# let's read the HAM10000_metadata.csv into the tile_df
tile_df = pd.read_csv('c_skin/HAM10000_metadata.csv')
tile_df.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.00,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.00,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.00,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.00,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.00,male,ear


In [23]:
# let's create a new column for the image path
tile_df['path'] = tile_df['image_id'].map(imageid_path_dict.get)
tile_df.sample(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization,path
9763,HAM_0006482,ISIC_0028372,akiec,histo,55.00,female,hand,c_skin/ham10000_images_part_1/ISIC_0028372.jpg
2071,HAM_0002523,ISIC_0024516,mel,histo,40.00,male,back,c_skin/ham10000_images_part_1/ISIC_0024516.jpg
5225,HAM_0004972,ISIC_0026997,nv,follow_up,50.00,male,lower extremity,c_skin/ham10000_images_part_1/ISIC_0026997.jpg
7094,HAM_0002581,ISIC_0032470,nv,histo,40.00,male,back,c_skin/HAM10000_images_part_2/ISIC_0032470.jpg
3433,HAM_0005797,ISIC_0027220,nv,follow_up,60.00,male,abdomen,c_skin/ham10000_images_part_1/ISIC_0027220.jpg


### 2.3 Create lesion type dictionary

In [0]:
# create dictionary of the different lesion types - this will be needed to index the skin lesin types numerically
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'Melanoma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [25]:
# create the cell_type_idx column, which represents the lesion type in numeric value
# later in the notebook the celltype_idx and and image features will be needed for the tf data pipeline
tile_df['cell_type'] = tile_df['dx'].map(lesion_type_dict.get) 
tile_df['cell_type_idx'] = pd.Categorical(tile_df['cell_type']).codes
tile_df.sample(5)

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
8445,HAM_0003698,ISIC_0026101,nv,histo,40.00,female,back,c_skin/ham10000_images_part_1/ISIC_0026101.jpg,Melanocytic nevi,4
9729,HAM_0007601,ISIC_0032437,akiec,histo,65.00,male,upper extremity,c_skin/HAM10000_images_part_2/ISIC_0032437.jpg,Actinic keratoses,0
2917,HAM_0002269,ISIC_0024443,bcc,histo,55.00,male,back,c_skin/ham10000_images_part_1/ISIC_0024443.jpg,Basal cell carcinoma,1
627,HAM_0002358,ISIC_0028650,bkl,histo,85.00,female,face,c_skin/ham10000_images_part_1/ISIC_0028650.jpg,Benign keratosis-like lesions,2
2586,HAM_0003328,ISIC_0026453,bcc,histo,55.00,female,back,c_skin/ham10000_images_part_1/ISIC_0026453.jpg,Basal cell carcinoma,1


### 2.4 Load images via path

In [0]:
# load all images via the path creating a new column for the image pixel data
from skimage.io import imread
tile_df['image'] = tile_df['path'].map(imread)

In [27]:
# let's check what the image column looks like
tile_df.head(6)

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx,image
0,HAM_0000118,ISIC_0027419,bkl,histo,80.00,male,scalp,c_skin/ham10000_images_part_1/ISIC_0027419.jpg,Benign keratosis-like lesions,2,"[[[188, 147, 191], [186, 148, 189], [187, 150,..."
1,HAM_0000118,ISIC_0025030,bkl,histo,80.00,male,scalp,c_skin/ham10000_images_part_1/ISIC_0025030.jpg,Benign keratosis-like lesions,2,"[[[25, 15, 23], [25, 14, 22], [25, 14, 22], [2..."
2,HAM_0002730,ISIC_0026769,bkl,histo,80.00,male,scalp,c_skin/ham10000_images_part_1/ISIC_0026769.jpg,Benign keratosis-like lesions,2,"[[[186, 128, 140], [188, 128, 136], [183, 126,..."
3,HAM_0002730,ISIC_0025661,bkl,histo,80.00,male,scalp,c_skin/ham10000_images_part_1/ISIC_0025661.jpg,Benign keratosis-like lesions,2,"[[[24, 9, 16], [22, 11, 15], [23, 11, 15], [26..."
4,HAM_0001466,ISIC_0031633,bkl,histo,75.00,male,ear,c_skin/HAM10000_images_part_2/ISIC_0031633.jpg,Benign keratosis-like lesions,2,"[[[122, 80, 102], [124, 82, 104], [127, 83, 10..."
5,HAM_0001466,ISIC_0027850,bkl,histo,75.00,male,ear,c_skin/ham10000_images_part_1/ISIC_0027850.jpg,Benign keratosis-like lesions,2,"[[[4, 0, 0], [4, 0, 0], [4, 0, 1], [4, 0, 1], ..."


In [28]:
# check the image size distribution
tile_df['image'].map(lambda x: x.shape).value_counts()

(450, 600, 3)    10015
Name: image, dtype: int64

In [29]:
# show the mapping between cell_type_idx and cell_type
tile_df[['cell_type_idx', 'cell_type']].sort_values('cell_type_idx').drop_duplicates()

,cell_type_idx,cell_type
9804,0,Actinic keratoses
2528,1,Basal cell carcinoma
1018,2,Benign keratosis-like lesions
1159,3,Dermatofibroma
7050,4,Melanocytic nevi
1622,5,Melanoma
2413,6,Vascular lesions


## 3. Data Transformation

### 3.1 Resizing images to 28x28

In [0]:
# resize the images scikit-images to a smaller scale of 28x28x3
from PIL import Image, ImageFont

tile_df['image'] = tile_df['path'].map(lambda x: np.asarray(Image.open(x).resize((28,28))))

In [31]:
# let's check the image size distribution again and see if the resizing worked 
tile_df['image'].map(lambda x: x.shape).value_counts()

(28, 28, 3)    10015
Name: image, dtype: int64

In [0]:
# save a copy of the preprocessed ham_dataset file to My Drive
#from google.colab import drive
#drive.mount('/drive')
#ham_df.to_csv('/drive/My Drive/Colab_Data/ham_10000_dataset_28x28.csv')

### 3.2 Select relevant data from df

In [41]:
# select only the relevant columns and rename the columns titles
tile_df = tile_df[['image','cell_type_idx']]

ham_dataset = tile_df.rename(columns={"image": "images", "cell_type_idx": "labels"})
ham_dataset.sample(6)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-41-e05b36d9187d>", line 1, in <module>
    tile_df = tile_df[['image','cell_type_idx']]
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py", line 2806, in __getitem__
    indexer = self.loc._get_listlike_indexer(key, axis=1, raise_missing=True)[1]
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py", line 1553, in _get_listlike_indexer
    keyarr, indexer, o._get_axis_number(axis), raise_missing=raise_missing
  File "/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py", line 1635, in _validate_read_indexer
    missing = (indexer < 0).sum()
  File "/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py", line 38, in _sum
    return umr_sum(a, axis, dtype, out, keepdims, initial, where)
TypeError: int() argument must be a

TypeError: ignored

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in callback <bound method AutoreloadMagics.post_execute_hook of <autoreload.AutoreloadMagics object at 0x7faa95254cf8>> (for post_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_event_manager.py", line 28, in trigger
    func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 524, in post_execute_hook
    _, pymtime = self._reloader.filename_and_mtime(sys.modules[modname])
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 186, in filename_and_mtime
    if not hasattr(module, '__file__') or module.__file__ is None:
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, 

ImportError: ignored

## 4. Train, Test, Split

### 4.1 Select features and target


In [0]:
import tensorflow as tf
from tensorflow import keras

In [34]:
features = ham_dataset.drop(columns=['labels'])
features.head()

,images
0,"[[[192, 153, 193], [195, 155, 192], [197, 154,..."
1,"[[[27, 16, 32], [69, 49, 76], [122, 93, 126], ..."
2,"[[[192, 138, 153], [200, 144, 162], [202, 142,..."
3,"[[[40, 21, 31], [95, 61, 73], [143, 102, 118],..."
4,"[[[159, 114, 140], [194, 144, 173], [215, 162,..."


In [35]:
target = ham_dataset['labels']
target.head()

0    2
1    2
2    2
3    2
4    2
Name: labels, dtype: int8

In [0]:
# split the dataset into training and testing set of 80:20 ratio
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

from sklearn.model_selection import train_test_split

x_train_o, x_test_o, y_train_o, y_test_o = train_test_split(features, target, test_size=0.20, random_state=10015)

## 5. Packaging dataset for TFF

Create a df with only the image(pixel) and the lable. The emnist data set in the federated example has the following `element_type_structure`

OrderedDict([('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None)), ('label', TensorSpec(shape=(), dtype=tf.int32, name=None))])

### 5.1 Installations

In [39]:
#@test {"skip": true}
!pip install --quiet --upgrade tensorflow_federated

# NOTE: Jupyter requires a patch to asyncio.
!pip install --quiet --upgrade nest_asyncio
import nest_asyncio
nest_asyncio.apply()

%load_ext tensorboard

     |████████████████████████████████| 430kB 3.5MB/s 
     |████████████████████████████████| 2.2MB 15.8MB/s 
     |████████████████████████████████| 296kB 34.5MB/s 
     |████████████████████████████████| 20.0MB 1.2MB/s 
     |████████████████████████████████| 2.8MB 61.1MB/s 
     |████████████████████████████████| 102kB 11.5MB/s 
     |████████████████████████████████| 421.8MB 34kB/s 
     |████████████████████████████████| 450kB 48.9MB/s 
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [40]:
import collections
import warnings
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

warnings.simplefilter('ignore')

tf.compat.v1.enable_v2_behavior()

np.random.seed(0)

tff.federated_computation(lambda: 'Hello, World!')()

[autoreload of numpy.core.multiarray failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
RuntimeError: empty_like method already has a docstring
]
[autoreload of numpy.core.overrides failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
RuntimeError: implement_array_function method already has a docstring
]
[autoreload of numpy.lib failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
NameError: name 'type_check' is not defined
]
[autoreload of numpy.lib.scimath failed: Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", l

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-40-1d768f6af4e4>", line 5, in <module>
    import tensorflow_federated as tff
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_federated/__init__.py", line 62, in <module>
    from tensorflow_federated.python import simulation
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_federated/python/simulation/__init__.py", line 17, in <module>
    from tensorflow_federated.python.simulation import datasets
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_federated/python/simulation/datasets/__init__.py", line 18, in <module>
    from tensorflow_federated.python.simulation.datasets import emnist
  File "/usr/local/lib/python3.6/dist-packages/tensorflow_federated/python/simulation/datasets/emnist.py", line 25, in <module>
    import tensorflow_addons.imag

ImportError: ignored

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Error in callback <bound method AutoreloadMagics.post_execute_hook of <autoreload.AutoreloadMagics object at 0x7faa95254cf8>> (for post_execute):
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/google/colab/_event_manager.py", line 28, in trigger
    func(*args, **kwargs)
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 524, in post_execute_hook
    _, pymtime = self._reloader.filename_and_mtime(sys.modules[modname])
  File "/usr/local/lib/python3.6/dist-packages/IPython/extensions/autoreload.py", line 186, in filename_and_mtime
    if not hasattr(module, '__file__') or module.__file__ is None:
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 50, in __getattr__
    module = self._load()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "/usr/lib/python3.6/importlib/__init__.py", line 126, 

ImportError: ignored

### 5.2 tf.data - build input pipelines

In [0]:
# Loading image data into tfds format
DATA_URL = 'https://drive.google.com/drive/folders/1ZoNx2IpYg0LI6xIjc8z5NuhE1bv4vhZh'
data_root_orig = tf.keras.utils.get_file(fname='ham_dataset_28x28',origin=DATA_URL,untar=True)
data_root = pathlib.Path(data_root_orig)

ERROR! Session/line number was not unique in database. History logging moved to new session 62
   8192/Unknown - 0s 0us/step

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((ham_dataset.images, ham_dataset.labels))

ERROR! Session/line number was not unique in database. History logging moved to new session 59


ValueError: ignored

### 5.3 tff.simulation.datasets.build_synthethic_iid_datasets

### 5.4 tff.simulation.ClientData

In [37]:
example_tff_dataset = x_train_o.create_tf_dataset_for_client(x_train_o.client_ids[0])

AttributeError: ignored